# Creating topics for the cases
This notebooks explores the topics we can create from the cases. It's gonna involve a lot of trial and error, but it will hopefully be well-documented

In [6]:
import pickle
import requests
import random
import pandas as pd
import numpy as np
from pathlib import Path
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
def read_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

def pickle_object(obj, file_path):
    with open(file_path, "wb") as f:
        pickle.dump(obj, f)

def flatten_embeddings(embedding_dict):
    """ Creates a big matrix with all the embeddings from the dict """
    return np.vstack(embedding_dict.values())

def flatten_list(lst):
    return [elem for sublist in lst for elem in sublist]

def get_paragraphs(paragraph_dict):
    return flatten_list(list(paragraph_dict.values()))


def load_text_url(text_url: str): 
    """Inputs a URL of a newline seperated text-file and returns a list"""
    response = requests.get(text_url)
    return response.text.split("\n")

## Loading the data

In [8]:
DATA_DIR = Path("../../BscThesisData/data")
MODEL_PATH = Path("../models")
embedding_dict = read_pickle(DATA_DIR / "embedding_dict.pkl")
clean_paragraphs = read_pickle(DATA_DIR / "paragraph_dict.pkl")

Now it's time to extract the embeddings for BERTopic to process

In [9]:
embeddings = flatten_embeddings(embedding_dict)
docs = get_paragraphs(clean_paragraphs)
print(random.sample(docs, 5))

['Appellanten havde ligeledes gjort en række formelle indsigelser gældende, herunder at skatteankenævnet ikke havde været berettiget til at ændre ansættelsesgrundlaget fra skatteforvaltningslovens § 27 til skatteforvaltningslovens § 26, stk. 5.', 'Sagen vedrørte i første række spørgsmålet om, hvorvidt sagsøgeren var skattepligtig i Danmark af beløb overført til hendes og hendes barns konto fra hendes tidligere samlever i Schweitz. I anden række angik sagen, om der var grundlag for genoptagelse som følge af grov uagtsomhed fra sagsøgeren, og om SKAT havde overholdt varslingsfristen for genoptagelsen.', 'De omtvistede kautionsforpligtelser og udgifter til mæglerhonorar og due diligence kunne derfor ikke indgå på As fradragskonto, hvorfor Skatteministeriet blev frifundet.', 'Retten anførte, at rentelovens § 8 a ikke indeholder hjemmel til at udbetale renter af tilbagebetalte sagsomkostninger. Krav på tilbagebetaling af sagsomkostninger forrentes i stedet som udgangspunkt efter den alminde

### Now we initialize the models

# Load Cleaning Models

In [10]:
STOP_WORD_URL = "https://gist.githubusercontent.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b/raw/305d8e3930cc419e909d49d4b489c9773f75b2d6/stopord.txt"
STOP_WORDS = load_text_url(STOP_WORD_URL)
vectorizer_model = CountVectorizer(stop_words=STOP_WORDS)
pickle_object(vectorizer_model, MODEL_PATH / "vectorizer.pkl")

In [11]:
# Powering up the transformer!
topic_model = BERTopic("Maltehb/-l-ctra-danish-electra-small-cased", vectorizer_model=vectorizer_model, nr_topics=5)

In [12]:
topics, probs = topic_model.fit_transform(docs, embeddings)

In [13]:
topic_model.visualize_topics()

In [14]:
preds_df = pd.DataFrame(list(zip(topics, probs, docs)), columns = ["topic", "prob", "doc"])
preds_df.to_csv(DATA_DIR / "doc_topics.csv", index=False)

In [15]:
# Saving the model
topic_model.save(str(MODEL_PATH / "topic_model"), save_embedding_model=False)

C:\Users\jhr\Anaconda3\envs\bertopic_explore\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

